# Implementing C/NC value

Method implemented based on <https://doi.org/10.1007/s007999900023>

The C-value approach combines linguistic and statistical information, emphasis being placed on the statistical part. The linguistic information consists of the part-of-speech tagging of the corpus, the linguistic filter constraining the type of terms extracted, and the stop-list. The statistical part combines statistical features of the candidate string, in a form of measure that is also called C-value.

$ C-value(a) = log_2 |a| \times f(a) $ if a is not nested,

$ C-value(a) = log_2 |a| \times (f(a)− \frac{1}{P(T_a)} \sum_{b \in T_a} f(b)) $ otherwise

Section 2.3 The Algorithm, page 3

In [1]:
import json
from random import sample
from nltk.corpus import stopwords
from nltk.util import ngrams
import re
from string import punctuation
from collections import Counter, defaultdict
import math
from pprint import pprint as pp

## Load data

In [2]:
data_path = "C:/Users/msesboue/OneDrive - TRACEPARTS/These_RESPONDING/Data/es_data_info/schneider_pn_texts/"
# data_file = "schneider_pn_fr_texts.csv"
data_file = "schneider_texts.csv"

In [3]:
with open(data_path + data_file, "r", encoding='utf8') as file:
    texts = list(set([text.strip('"\n') for text in file.readlines()]))
print(len(texts))

57395


In [4]:
sample_size = 10000
sample_docs = [txt.lower().split() for txt in sample(texts, sample_size)]
# sample(sample_docs, 10)

## Preprocessing

This preprocessing step is very specific to my data.

TODO:

- When generating tokens. If some tokens have been removed in between two sequences, no ngram should be composed of elements of those two separate sequences.

In [5]:
only_letters_pattern = re.compile(r'[a-z]+')

In [6]:
docs = [[token.strip(punctuation) for token in doc] for doc in sample_docs]
docs = [[token for token in doc if (len(token) > 0)] for doc in docs]

In [7]:
corpus = dict()
for i, txt in enumerate(docs):
    corpus[i] = txt

## Filter the type of terms

add filter by occurence ?

*Step 1 : We tag the corpus. As mentioned earlier, we need the tagging process since we will use a linguistic filter to restrict the type of terms to be extracted.*

*Step 2: This stage extracts those strings that satisfy the linguistic filter and occurence threshold.*

In this case we keep tokens composed of only letters and remove stop words. 

In [8]:
only_letters_pattern = re.compile(r'[a-z]+')
en_stopwords = stopwords.words('english')

empty_docs = []

for idx, tokens in corpus.items():
    corpus[idx] = [t for t in tokens if (only_letters_pattern.fullmatch(t) is not None) and (t not in en_stopwords)]
    
    if len(corpus[idx]) == 0:     
        empty_docs.append(idx)
        
for idx in empty_docs:
    del corpus[idx]

## Create N-grams

*The lists are then filtered through the stop-list and are concatenated. The longest strings appear at the top, and decrease in size as we move down, with the bigrams being at the bottom. The strings of each length are ordered by their occurrence.*

In [9]:
max_size_gram = 5 # need to be chosen

grams_by_size = defaultdict(list)
all_ngrams = []

for size in range(1, max_size_gram + 1):
    for tokens in corpus.values():
        grams_by_size[size].extend([" ".join(gram) for gram in ngrams(tokens, size)])
    
    all_ngrams.extend(grams_by_size[size])
    # we need all ngrams (with duplicates) only to extract the occurence
    # for the we will keep a list of unique ngram instances
    grams_by_size[size] = list(set(grams_by_size[size])) 

In [10]:
ngram_occurences = Counter(all_ngrams) # The occurence of each ngram in the corpus

# each group of ngram needs to be ordered by the occurence
for size in grams_by_size.keys():
    grams_by_size[size].sort(key=lambda ngram: ngram_occurences[ngram], reverse=True)
    
unique_ngrams = []
for size in range(1, max_size_gram + 1).__reversed__():
    unique_ngrams.extend(grams_by_size[size])

## Compute C-value

In the paper, there is mistake in the last if condition alignement. It should be out of the above else.

In [11]:
def get_substrings(string):
    tokens = string.split()
    token_len = len(tokens)
    
    substrings = set()
    for i in range(1, token_len):
        for gram in ngrams(tokens, i):
            substrings.add(" ".join(gram))
    
    return list(substrings)

def update_stat_triple(substring, stat_triples, parent_str, counter):
    if substring in stat_triples.keys():
        if parent_str in stat_triples.keys():
            stat_triples[substring][1] = stat_triples[substring][1] + (counter[parent_str] - stat_triples[parent_str][1])
        else:
            stat_triples[substring][1] = stat_triples[substring][1] + counter[parent_str]
            
        stat_triples[substring][2] += 1
    else:
        f_string = 0 if counter.get(substring) is None else counter[substring]
        stat_triples[substring] = [f_string, counter[parent_str], 1]
        
def process_substrings(ngram, stat_triples, counter):
    substrings = get_substrings(ngram)
    for substring in substrings:
        update_stat_triple(substring, stat_triples, ngram, counter)
        
def computes_c_values(ngrams, counter, max_size_gram):
    
    c_values = list()
    stat_triples = dict()

    for ngram in ngrams:
        
        len_ngram = len(ngram.split())
        
        if len_ngram == max_size_gram:
            c_val = math.log2(len_ngram) * counter[ngram]
            c_values.append((c_val, ngram))
        
            process_substrings(ngram, stat_triples, counter)
        
        else:
            if ngram not in stat_triples.keys():
                c_val = math.log2(len_ngram) * counter[ngram]
                c_values.append((c_val, ngram))
            else:
                c_val = math.log2(len_ngram) * (counter[ngram] - (stat_triples[ngram][1] / stat_triples[ngram][2]))
                c_values.append((c_val, ngram))
                
            process_substrings(ngram, stat_triples, counter)
    
    return c_values

In [12]:
c_values = computes_c_values(unique_ngrams, ngram_occurences, 5)

c_values.sort(key=lambda c_val: c_val[0], reverse=True)

## Watch results

In [13]:
c_values[:10]

[(4260.738054118309, 'kw power base control unit'),
 (4228.231060789886, 'base control unit communication module'),
 (4228.231060789886, 'power base control unit communication'),
 (3139.2467842877136, 'control unit communication module accessories'),
 (2492.9195904199896, 'base control unit'),
 (2326.724951058657, 'power base control'),
 (2308.97337105058, 'control unit communication'),
 (2164.662535359919, 'unit communication module'),
 (1797.5294117647059, 'control unit'),
 (1722.6666666666667, 'communication module')]

## Test

Example from the paper. 

| C-value | P(Ta) | SUM(f(b)) | Freq. | Candidate Terms |
| --- | --- | --- | --- | --- |
| 11.6096 | 0 | 0 | 5 | ADENOID CYSTIC BASAL CELL CARCINOMA |
| 12 | 5 | 1 | 11 | CYSTIC BASAL CELL CARCINOMA |
| 14 | 0 | 0 | 7 | ULCERATED BASAL CELL CARCINOMA |
| 10 | 0 | 0 | 5 | RECURRENT BASAL CELL CARCINOMA |
| 6 | 0 | 0 | 3 | CIRCUMSCRIBED BASAL CELL CARCINOMA |
| 1551.36 | 26 | 5 | 984 | BASAL CELL CARCINOMA |

In [14]:
test_ngrams = [
    "ADENOID CYSTIC BASAL CELL CARCINOMA",
    "CYSTIC BASAL CELL CARCINOMA",
    "ULCERATED BASAL CELL CARCINOMA",
    "RECURRENT BASAL CELL CARCINOMA",
    "CIRCUMSCRIBED BASAL CELL CARCINOMA",
    "BASAL CELL CARCINOMA"
]

test_counter = {
    "ADENOID CYSTIC BASAL CELL CARCINOMA": 5,
    "CYSTIC BASAL CELL CARCINOMA": 11,
    "ULCERATED BASAL CELL CARCINOMA": 7,
    "RECURRENT BASAL CELL CARCINOMA": 5,
    "CIRCUMSCRIBED BASAL CELL CARCINOMA": 3,
    "BASAL CELL CARCINOMA": 984
}


test_c_values = computes_c_values(test_ngrams, test_counter, 5)          

pp(test_c_values)

[(11.60964047443681, 'ADENOID CYSTIC BASAL CELL CARCINOMA'),
 (12.0, 'CYSTIC BASAL CELL CARCINOMA'),
 (14.0, 'ULCERATED BASAL CELL CARCINOMA'),
 (10.0, 'RECURRENT BASAL CELL CARCINOMA'),
 (6.0, 'CIRCUMSCRIBED BASAL CELL CARCINOMA'),
 (1551.3612957058674, 'BASAL CELL CARCINOMA')]
